In [2]:
### Get all key dfs from Cleaned Data
import pandas as pd

df_t = pd.read_csv('Data/Cleaned/TSS metrics.csv')
df_s = pd.read_csv('Data/Cleaned/Sleep_and_recovery.csv')
df_f = pd.read_csv('Data/Cleaned/MFP per day scrapped.csv')
df_g = pd.read_csv('Data/Cleaned/Glucose_daily.csv')

# Print the min and the max date of each df
print('TSS metrics: ', df_t['date'].min(),' to ',df_t['date'].max())
print('Sleep and recovery: ',df_s['date'].min(),' to ',df_s['date'].max())
print('MFP per day scrapped: ',df_f['date'].min(),' to ',df_f['date'].max())
print('Glucose daily: ',df_g['date'].min(),' to ',df_g['date'].max())


TSS metrics:  2023-04-28  to  2024-04-27
Sleep and recovery:  2024-03-16  to  2024-04-27
MFP per day scrapped:  2024-03-16  to  2024-04-26
Glucose daily:  2024-03-23  to  2024-04-27


In [4]:
# Filter all dfs to the same date range, the one between the min and max date of all dfs
min_date = max(df_t['date'].min(),df_s['date'].min(),df_f['date'].min(),df_g['date'].min())
max_date = min(df_t['date'].max(),df_s['date'].max(),df_f['date'].max(),df_g['date'].max())

df_t = df_t[(df_t['date'] >= min_date) & (df_t['date'] <= max_date)]
df_s = df_s[(df_s['date'] >= min_date) & (df_s['date'] <= max_date)]
df_f = df_f[(df_f['date'] >= min_date) & (df_f['date'] <= max_date)]
df_g = df_g[(df_g['date'] >= min_date) & (df_g['date'] <= max_date)]

# Perform an outter join on all dfs
df = df_t.merge(df_s, on='date', how='outer')
df = df.merge(df_f, on='date', how='outer')
df = df.merge(df_g, on='date', how='outer')

# Check first and last date of the new df
print('New df: ',df['date'].min(),' to ',df['date'].max())

# Drop sleep id
df.drop('sleep_id',axis=1,inplace=True)

df.columns

New df:  2024-03-23  to  2024-04-26


Index(['date', 'TSS', 'CTL', 'ATL', 'TSB', 'sleep_time',
       'sleep_score_performance', 'sleep_score_consistency',
       'sleep_score_efficiency', 'sleep_duration', 'sleep_rem', 'sleep_deep',
       'sleep_light', 'sleep_awake', 'sleep_unspecified', 'recovery_score',
       'resting_hr', 'hrv', 'spo2', 'skin_temp', 'calories_burned', 'carbs',
       'fat', 'protein', 'sodium', 'sugar', 'calories_consumed',
       'calories_goal', 'calories_net', 'calories_consumed_breakfast',
       'calories_consumed_lunch', 'calories_consumed_dinner',
       'calories_consumed_snacks', 'mean_glucose', 'std_glucose',
       'max_glucose', 'wake_up_glucose'],
      dtype='object')

In [55]:
# Save to CSV
df.to_csv('Data/Cleaned/Integrated_data.csv', index=False)

In [5]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Load service account credentials
creds = service_account.Credentials.from_service_account_file('gsheets key.json')
scoped_credentials = creds.with_scopes(['https://www.googleapis.com/auth/spreadsheets'])

# Authenticate with Google Sheets API
service = build('sheets', 'v4', credentials=scoped_credentials)

# Spreadsheet ID and range for Health Dashboards
SPREADSHEET_ID_input = '197VfZCekvBev0m1vsi8kUHpuO0IoTRA90_bQRGBYYSM'

def Export_Data_To_Sheets(df, sheet_name):
    # Specify the range including the sheet name
    range_name = f"{sheet_name}!A1:ZA1000"
    
    # Fill NaN values with empty strings
    df_filled = df.fillna('')
    
    # Get DataFrame headers and values
    headers = [df_filled.columns.tolist()]
    values = df_filled.values.tolist()
    
    # Concatenate headers with values
    data = headers + values
    
    # Update the spreadsheet with DataFrame values including headers
    response = service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID_input,
        valueInputOption='RAW',
        range=range_name,
        body=dict(
            majorDimension='ROWS',
            values=data)
    ).execute()

# Specify the sheet name
sheet_name = 'Integrated_data'

# Export DataFrame to Google Sheets with specified sheet name
Export_Data_To_Sheets(df, sheet_name)
